# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [27]:
import pandas as pd
import numpy as np

data = pd.read_csv('/datasets/data.csv')
data.info() #общая информация о  таблице
display(data.head(10)) #для наглядности, выведем первые 10 строк

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.1 Заполнение пропусков


In [28]:
print('Количество пустых строк:') 
print(data.isnull().sum()) #проверяем количество пустых строк и столбцы с пустыми строками
print('Общая длина таблицы:', data.shape[0]) #находим длину таблицы
print('Процент пропусков: {:.2%}'.format(data['total_income'].isnull().sum() / data.shape[0])) #находим процент пропущенных значений
data['total_income'] = data['total_income'].fillna(data['total_income'].median()) #заполняем пустые строки медианным значением 
print(data.isnull().sum()) #проверяем данные

Количество пустых строк:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Общая длина таблицы: 21525
Процент пропусков: 10.10%
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


Если обратить внимание на названия столбцов, которые имеют пустые строки, можно  заметить, что количество пустых строк в обоих столбцах одинаково. Исходя из названий столбцов можно сделать вывод, что пустые значения заработка находятся там, где у человека отсутствует трудовой стаж.
Заполнение пропусков медианным значением обусловлено тем, что среди набора данных могут встретиться данные, которые сильно отличаются от большинства.


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [29]:
display(data[data['income_type'] == 'пенсионер']['days_employed']) #все значения трудового стажа, где вид профессии - пенсионер, имеют слишком большое значение
display(data[data['income_type'] != 'пенсионер']['days_employed']) #оставшиеся значения трудовго стажа - отрицательные
print()
print(data['children'].unique()) #проверяем ошибки в столбце "дети"
print()
print(data['dob_years'].unique()) #проверяем ошибки в столбце "возраст"
print()
print(data[data['total_income'] < 0]['total_income']) #проверяем на наличие отрицательного заработка

4        340266.072047
12                 NaN
18       400281.136913
24       338551.952911
25       363548.489348
             ...      
21505    338904.866406
21508    386497.714078
21509    362161.054124
21518    373995.710838
21521    343937.404131
Name: days_employed, Length: 3856, dtype: float64

0       -8437.673028
1       -4024.803754
2       -5623.422610
3       -4124.747207
5        -926.185831
            ...     
21519   -2351.431934
21520   -4529.316663
21522   -2113.346888
21523   -3112.481705
21524   -1984.507589
Name: days_employed, Length: 17669, dtype: float64


[ 1  0  3  2 -1  4 20  5]

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

Series([], Name: total_income, dtype: float64)


Видим, что кроме аномалий в столбце "трудовой стаж", имеются аномалии в столбце "дети" (отрицательное количество детей и 20 детей) и в столбце "возраст" (0 лет).
В теории ошибки в количстве детей могут быть связаны с человеческим фактором: человек написал -1 вместо 1 или 20 вместо 2, но я не верю в это, слишком уж много аномальных значений получилось.  Поэтому просто заменим эти значения на медианные.

Ошибки в трудовом стаже: для пенсионеров - значения указаны в часах, для остальных - ошибка в знаке.

Ошибку в возрасте исправим, заменив "0" на медиану.

In [30]:
data[data.duplicated(keep=False) & (data['income_type'] == 'сотрудник')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
120,0,NaN,46,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,высшее образование
520,0,NaN,35,среднее,1,гражданский брак,1,F,сотрудник,0,145017.937533,сыграть свадьбу
541,0,NaN,57,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,сделка с подержанным автомобилем
554,0,NaN,60,среднее,1,женат / замужем,0,M,сотрудник,0,145017.937533,покупка недвижимости
1431,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,покупка жилья для семьи
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,покупка жилья для семьи
3111,0,NaN,54,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,операции со своей недвижимостью
3452,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017.937533,покупка жилой недвижимости
3690,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017.937533,свадьба
4182,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017.937533,свадьба


In [31]:
# Код ревьюера

data['days_employed'].isna().sum()

2174

In [32]:
data.loc[data['income_type'] == 'пенсионер', 'days_employed'] = data.loc[data['income_type'] == 'пенсионер', 'days_employed'] / 24 #заменяем все значения стажа, где вид деятельности - пенсионер (разделим на 24 часа)
data['days_employed'] = abs(data['days_employed']) #заменяем все отрицательные значения в столбце days_employed на положительные
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
display(data.head()) #проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [33]:
# Код ревьюера

data['days_employed'].isna().sum()

0

In [34]:
data['dob_years'] = data['dob_years'].replace(0, np.nan) 
data['dob_years'] = data['dob_years'].fillna(data['dob_years'].median()) #заменяем количество лет равное 0 на NaN, а затем на медианное значение
data['dob_years'] = data['dob_years'].astype(int) #переводим в целые числа
print(data['dob_years'].unique()) #проверяем

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


In [35]:
display(data[data['children'] == -1])
print()
display(data[data['children'] == 20]) #смотрим с чем могут быть связаны аномалии в количестве детей

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,14582.827176,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,2194.220567,57,Среднее,1,женат / замужем,0,F,пенсионер,0,145017.937533,на покупку своего автомобиля
1363,-1,1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,2161.591519,43,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,2194.220567,53,среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,покупка жилой недвижимости
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


In [36]:
data['children'] = data['children'].replace([data[data['children'] == -1]['children']], np.nan)
data['children'] = data['children'].replace([data[data['children'] == 20]['children']], np.nan)
data['children'] = data['children'].fillna(data['children'].median()) #заменяем количество детей равное -1 и 20 на NaN, а затем на медианное значение
data['children'] = data['children'].astype(int)
print(data['children'].unique())

[1 0 3 2 4 5]


### Шаг 2.3. Изменение типов данных.

In [37]:
data.info()
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------    

### Шаг 2.4. Удаление дубликатов.

In [38]:
print(data.duplicated().sum()) #проверяем на явные дубликаты

54


Для каждого столбца типа object уникальные значения и их количество.

In [39]:
display(data['education'].value_counts())  
print()
display(data['family_status'].value_counts())
print()
display(data['gender'].value_counts())
print()
display(data['income_type'].value_counts())
print()
display(data['purpose'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [40]:
data['education'] = data['education'].str.lower() #переводим все значения столбца "образование" в нижний регистр
print(data.duplicated().sum())#проверяем количество дубликатов
data = data.drop_duplicates().reset_index(drop = True)#удаляем их
print(data.duplicated().sum())

71
0


Причиной появления дубликатов может быть повторная подача заявления на кредит от того же человека.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [41]:
education_dict = data[['education_id', 'education']] #создаем датафрейм-словарь
education_dict = education_dict.drop_duplicates().reset_index(drop = True)#удаляем дубликаты
display(education_dict)
family_dict = data[['family_status_id', 'family_status']]#повторяем для других столбцов
family_dict = family_dict.drop_duplicates().reset_index(drop = True)
display(family_dict)


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [42]:
data = data.drop(['education', 'family_status'], axis = 1) #удаляем избыточные столбцы из основной таблицы
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [43]:
def income_category(inc): #создаем функцию для определения категории дохода
    if inc <= 30000:
        return 'E'
    elif inc > 30000 and inc <= 50000:
        return 'D'
    elif inc > 50000 and inc <= 200000:
        return 'C'
    elif inc > 200000 and inc <= 1000000:
        return 'B'
    else:
        return 'A'

data['total_income_category'] = data['total_income'].apply(income_category) #применяем функцию к каждой строчке столбца total_income
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [44]:
def purpose_category(purp):
    a = [['на проведение свадьбы', 'сыграть свадьбу'], 
          ['покупка коммерческой недвижимости', 'покупка жилья для сдачи', 'операции с жильем', 'операции с коммерческой недвижимостью',
          'покупка жилья', 'жилье', 'покупка жилья для семьи', 'строительство собственной недвижимости', 'операции с недвижимостью',
          'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости', 'покупка своего жилья',
          'строительство недвижимости', 'ремонт жилью', 'покупка жилой недвижимости'],
          ['на покупку своего автомобиля', 'сделка с подержанным автомобилем', 'свой автомобиль', 'на покупку подержанного автомобиля',
          'автомобили', 'на покупку автомобиля', 'приобретение автомобиля', 'сделка с автомобилем'],
         ['заняться высшим образованием', 'дополнительное образование', 'высшее образование', 'получение дополнительного образования',
         'получение образования', 'профильное образование', 'получение высшего образования', 'заняться образованием']] #список списков для замены значений
    for i in a:
        if i == a[0] and purp in i:#проверяем значение счетчика, если оно равно одному из значений списка списков, заменяем значения из подсписка на группирующее значение
            return 'проведение свадьбы' 
        elif i == a[1] and purp in i:
            return 'операции с недвижимостью'
        elif i == a[2] and purp in i:
            return 'операции с автомобилем'
        elif i == a[3] and purp in i:
            return 'получение образования'

data['purpose_category'] = data['purpose'].apply(purpose_category) #применяем функцию к каждой строчке столбца purpose
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [45]:
print('Процентное соотношение между количеством детей в семьях, где есть долг:')
display(data[data['debt'] == 1].groupby('children')['debt'].count() / data.groupby('children')['debt'].count() * 100)
print('Процентное соотношение между количеством детей в семьях, где нет долга:')
display(data[data['debt'] == 0].groupby('children')['debt'].count() / data.groupby('children')['debt'].count() * 100)



Процентное соотношение между количеством детей в семьях, где есть долг:


children
0    7.541860
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5         NaN
Name: debt, dtype: float64

Процентное соотношение между количеством детей в семьях, где нет долга:


children
0     92.458140
1     90.765391
2     90.545809
3     91.818182
4     90.243902
5    100.000000
Name: debt, dtype: float64

##### Вывод 1:

Процентные соотношения практически одинаковые, но все таки чем больше детей, тем выше шанс неуплаты. Под вопросом остаются данные по людям с 3 детьми.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:


display(data[data['debt'] == 1].groupby('family_status_id')['debt'].count() / data.groupby('family_status_id')['debt'].count() * 100) #процентное соотношение для должников
display(data[data['debt'] == 0].groupby('family_status_id')['debt'].count() / data.groupby('family_status_id')['debt'].count() * 100) #процентное соотношение для погасивших в срок
display(family_dict)

family_status_id
0    7.545182
1    9.347145
2    6.569343
3    7.112971
4    9.750890
Name: debt, dtype: float64

family_status_id
0    92.454818
1    90.652855
2    93.430657
3    92.887029
4    90.249110
Name: debt, dtype: float64

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


##### Вывод 2:

Люди из категории "женатые/замужние", "вдовец / вдова", "в разводе" реже имеют долг по кредиту, в отличие от холостых и людей в гражданском браке, которые наоборот неуплачивают чаще.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
display(data[data['debt'] == 1].groupby('total_income_category')['debt'].count() / data.groupby('total_income_category')['debt'].count() * 100) #процентное соотношение для должников
display(data[data['debt'] == 0].groupby('total_income_category')['debt'].count() / data.groupby('total_income_category')['debt'].count() * 100) #процентное соотношение для погасивших в срок


total_income_category
A    8.000000
B    7.062091
C    8.491508
D    6.000000
E    9.090909
Name: debt, dtype: float64

total_income_category
A    92.000000
B    92.937909
C    91.508492
D    94.000000
E    90.909091
Name: debt, dtype: float64

##### Вывод 3:

Классы С и Е чаще невыплачивает кредиты, а люди с доходом выше и чуть ниже среднего чаще погашает в срок. Зависимости возврата кредита в срок от уровня дохода не наблюдается.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [50]:
display(data[data['debt'] == 1].groupby('purpose_category')['debt'].count() / data.groupby('purpose_category')['debt'].count() * 100) #процентное соотношение для должников
display(data[data['debt'] == 0].groupby('purpose_category')['debt'].count() / data.groupby('purpose_category')['debt'].count() * 100) #процентное соотношение для погасивших в срок

purpose_category
операции с автомобилем      9.470094
операции с недвижимостью    7.270584
получение образования       9.478407
проведение свадьбы          7.958252
Name: debt, dtype: float64

purpose_category
операции с автомобилем      90.529906
операции с недвижимостью    92.729416
получение образования       90.521593
проведение свадьбы          92.041748
Name: debt, dtype: float64

##### Вывод 4:

Люди, целью кредита которых является операция с автомобилем или получение образования, чаще не возвращают кредиты в срок. Операции с недвижимостью и проведение свадьбы в свою очередь наоборот имеют выше процент возврата кредита.

## Общий вывод:

Чтобы определить потенциального неплатильщика, можно обратить внимание на количество детей, семейное положение, доход и цель заемщика. По статистике не возращают кредиты те люди, у которых: а) много детей, б) средний или низкий уровень дохода, в) холосты или находятся в гражданском браке, г) берут кредит для получения образования или проведения операции с автомобилем.